In [7]:
import cv2
import numpy as np
import cvlib as cv
from contextlib import contextmanager
from keras.utils.data_utils import get_file
from model import get_model



In [8]:
cap = cv2.VideoCapture(0)

In [9]:
#cap.read（）返回一個bool（True / False） ret回傳bool image 回傳畫面
while True:
   
    
    ret, image = cap.read()
    faces, confidences = cv.detect_face(image)
    
    model_name = "ResNet50"
    weight_file = "C:/Users/USER/Desktop/age-gender-estimation-master/age_estimation/age_only_resnet50_weights.061-3.300-4.410.hdf5"
    
    # load model and weights
    model = get_model(model_name=model_name)
    model.load_weights(weight_file)
    img_size = model.input.shape.as_list()[1]
    
    for i, (x, y, w, h ) in enumerate(faces):
        cv2.rectangle(image, (x, y), (w, h), (255, 255, 0), 2)
        # 將圖片轉換為 np.array，並作為模型的推論輸入
        #blob = cv2.dnn.blobFromImage(image, 1, (227, 227), MODEL_MEAN_VALUES, swapRB=False)
        
        #裁切臉龐
        face_crop = np.copy(image[y:h, x:w])
        #性別預測
        label, confidence = cv.detect_gender(face_crop)
        label = label[confidence.argmax()]
        conf = np.argmax(confidence)
        print("Gender : " + label+" {:.2f}%".format(confidence[conf] * 100))
        
        #年紀預測
        face = np.empty((len(faces), img_size, img_size, 3))
        face[i, :, :, :] = cv2.resize(image[y:h + 1, x:w + 1, :], (img_size, img_size))
        
        results = model.predict(face)
        ages = np.arange(0, 101).reshape(101, 1)
        predicted_ages = results.dot(ages).flatten()
        print("Age Range: " + "{}".format(int(predicted_ages[i])))
         
        
        overlay_text ="{:.2f}% ".format(confidences[0]* 100)+ "%s" % label  +" {}".format(int(predicted_ages[i]))
        #cv2.putText(影像, 文字, 座標, 字型, 大小, 顏色, 線條寬度, 線條種類)
        cv2.putText(image, overlay_text, (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
    cv2.imshow('frame', image)
    key = cv2.waitKey(60) 
    if key == 27:  #ESC
        # 釋放攝影機
        cap.release()
        # 關閉所有 OpenCV 視窗
        cv2.destroyAllWindows()
        break


Instructions for updating:
Colocations handled automatically by placer.
[INFO] initializing ... 
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Gender : man 99.05%
Age Range: 19
Gender : man 99.81%
Age Range: 21
Gender : man 99.68%
Age Range: 15
